<a href="https://colab.research.google.com/github/martinbremm/embedding-comparison/blob/main/BERT_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [19]:
!pip install transformers

In [3]:
import torch
from transformers import BertTokenizer, BertModel

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/Datasets

/content/drive/MyDrive/Datasets


# Importing and preprocessing data

In [51]:
import pandas as pd

df = pd.read_csv("data_frame_processed_labeled.csv")
# filtering out missing values in descriptions column
df = df[["clean_description", "clean_fn"]].dropna()

In [7]:
# examples of semantically- and acoustically-similar sound-producing mechanisms

# acoustically similar sound-producing mechanism
acoustic_dict = {
    "impact": ["bump", "drop", "fall", "bounce", "clap", "slam", "clang", "step", "punch"],
    "explosion": ["blast", "explode", "bang", "blowout", "burst"],
    "vocal": ["talk", "scream", "hiss", "yell", "chant", "conversation", "murmur", "shout"],
    "nose": ["wheeze", "sneeze", "snort", "snore", "sniff"],
}

# semantically similar sound-producing mechanism
semantic_dict = {
    "locomotion": ["walk", "gallop", "run" , "march", "jump", "move"],
    "dog": ["howl", "bark", "growl", "whimper", "scratch", "pant"],
    "vehicle": ["rev", "drive", "pass", "accelerate", "brake"],
    "complex_sound" : ["fight", "box", "destruction", "accident", "wrestle", "bowl", "exercise"]
}

In [8]:
def create_tuples(dictionary):
    dictionary_words = [word for words in dictionary.values() for word in words]  # list of words used as index
        
    category = []
    for key, val in dictionary.items():
        category.extend([key]*len(val)) # list of category memberships

    similarity = ["acoustic_sim" if dictionary=="acoustic_sim" else "semantic_sim" for word in dictionary_words]
        
    tuples = list(zip(similarity, category, dictionary_words))
    return tuples

In [10]:
acoustic_df = pd.DataFrame(data=create_tuples(acoustic_dict), columns = ["Similarity", "Category", "Word"])
semantic_df = pd.DataFrame(data=create_tuples(semantic_dict), columns = ["Similarity", "Category", "Word"])


combined_df = pd.concat([acoustic_df, semantic_df], ignore_index = True)

In [11]:
combined_df

,Similarity,Category,Word
0,semantic_sim,impact,bump
1,semantic_sim,impact,drop
2,semantic_sim,impact,fall
3,semantic_sim,impact,bounce
4,semantic_sim,impact,clap
5,semantic_sim,impact,slam
6,semantic_sim,impact,clang
7,semantic_sim,impact,step
8,semantic_sim,impact,punch
9,semantic_sim,explosion,blast


In [49]:
# tokenizing the sentences into single words
from nltk.tokenize import word_tokenize
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [52]:
tokens = df["clean_description"].apply(word_tokenize)
content = []
for word in combined_df["Word"]:
  word_df = df[tokens.apply(lambda sentence: word in sentence)]
  word_df.insert(0, "Word", [word]*len(word_df))
  word_df = word_df.reset_index(drop=True)
  content.append(word_df)
examples1 = pd.concat(content, ignore_index = True)

In [37]:
examples1

,Word,clean_description,clean_fn
0,bump,"[underwater, bump, bridge, mono, drops, falls,...",hit_concrete
1,bump,"[pinball, machine, hitting, knock, bump, amuse...",pinball_machine
2,bump,"[pinball, machine, hitting, knock, bump, hard,...",pinball_machine
3,bump,"[auto, crash, light, bump, hard, object, car, ...",car_crash
4,bump,"[auto, crash, light, bump, hard, object, car, ...",car_crash
...,...,...,...
74523,exercise,"[metal, drop, weight, plate, exercise, sport, ...",metal_drop
74524,exercise,"[metal, drop, weight, plate, exercise, sport, ...",metal_drop
74525,exercise,"[metal, drop, weight, plate, exercise, sport, ...",metal_drop
74526,exercise,"[metal, drop, weight, plate, exercise, sport, ...",metal_drop


In [53]:
content = []
for word in combined_df["Word"]:
  word_df = df[df["clean_description"].str.contains(" " + word + " ")]
  word_df.insert(0, "Word", [word]*len(word_df))
  word_df = word_df.reset_index(drop=True)
  content.append(word_df)
examples2 = pd.concat(content, ignore_index = True)

In [56]:
content = []
for word in combined_df["Word"]:
  word_df = df[df["clean_fn"].str.contains(word)]
  word_df.insert(0, "Word", [word]*len(word_df))
  word_df = word_df.reset_index(drop=True)
  content.append(word_df)
print(f"Contains examples of {len(content)} words!")

Contains examples of 51 words!


In [57]:
examples = pd.concat(content, ignore_index = True)
examples.to_csv("BERT_example_sentences.csv")

In [58]:
examples

,Word,clean_description,clean_fn
0,bump,wood en plank skid drag along bumpy surface,skid_wood_wooden_plank_drag_along_bumpy
1,bump,wood en plank skid drag along bumpy surface,skid_wood_wooden_plank_drag_along_bumpy
2,bump,video game electronic bump,video_game_electronic_bump
3,bump,luggage rolling bumps stop,luggage_rolling_bump
4,bump,body bumps wall various,body_bumps_wall_var
...,...,...,...
57617,exercise,exercise equipment weight lifting competition ...,exercise_equipment
57618,exercise,exercise equipment weight lifting competition ...,exercise_equipment
57619,exercise,exercise weight room training lifting bars wei...,exercise_weight_room
57620,exercise,exercise weight room training lifting bars wei...,exercise_weight_room


Creating Dataframe with example sentences obtained from out dataset (randomly chosen sentences including the words defined in the dictionaries)

In [ ]:
content = [(df[df["clean_description"].str.contains(" " + word + " ")].sample(n=1, ignore_index=True)[0]) for word in combined_df["Word"]]

combined_df["example_sentence"] = content

In [ ]:
combined_df.head(n=5)

,Similarity,Category,Word,example_sentence
0,semantic_sim,impact,bump,hits low bump impact
1,semantic_sim,impact,drop,truck door drop ground
2,semantic_sim,impact,fall,body falls fall human weeds
3,semantic_sim,impact,bounce,theater type folding seat bounce open close
4,semantic_sim,impact,clap,applause crowd small studio audience female cl...


# BERT model preprocessing

In [ ]:
# Adding BERT tokens
marked_text = combined_df["example_sentence"].apply(lambda sen: "[CLS] " + sen + " [SEP]")
marked_text

0                      [CLS] hits low bump impact [SEP]
1                    [CLS] truck door drop ground [SEP]
2               [CLS] body falls fall human weeds [SEP]
3     [CLS] theater type folding seat bounce open cl...
4     [CLS] applause crowd small studio audience fem...
5                    [CLS] porsche hood slam shut [SEP]
6     [CLS] metal thuds clang ringing knife ching qu...
7                 [CLS] ladder step climb ladders [SEP]
8     [CLS] classic old movie face punch single hit ...
9     [CLS] auto bmw horn short blast close horns ca...
10    [CLS] background ds munich new years celebrati...
11    [CLS] german precision metal ratchet clink cru...
12    [CLS] noisemaker paper blowout horn party comi...
13                     [CLS] zaps short burst zap [SEP]
14    [CLS] animals horses horse vocals stallion stu...
15    [CLS] human scream teenage girl screams female...
16    [CLS] tape noise microcassette hiss high speed...
17    [CLS] siren english police constant stop n

In [ ]:
## Splitting sentence into tokens

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenized_text = marked_text.apply(lambda sen: tokenizer.tokenize(sen))
tokenized_text

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

0               [[CLS], hits, low, bump, impact, [SEP]]
1             [[CLS], truck, door, drop, ground, [SEP]]
2       [[CLS], body, falls, fall, human, weeds, [SEP]]
3     [[CLS], theater, type, folding, seat, bounce, ...
4     [[CLS], applause, crowd, small, studio, audien...
5             [[CLS], porsche, hood, slam, shut, [SEP]]
6     [[CLS], metal, thud, ##s, clan, ##g, ringing, ...
7      [[CLS], ladder, step, climb, ladder, ##s, [SEP]]
8     [[CLS], classic, old, movie, face, punch, sing...
9     [[CLS], auto, bmw, horn, short, blast, close, ...
10    [[CLS], background, ds, munich, new, years, ce...
11    [[CLS], german, precision, metal, rat, ##chet,...
12    [[CLS], noise, ##maker, paper, blow, ##out, ho...
13      [[CLS], za, ##ps, short, burst, za, ##p, [SEP]]
14    [[CLS], animals, horses, horse, vocals, stalli...
15    [[CLS], human, scream, teenage, girl, screams,...
16    [[CLS], tape, noise, micro, ##cas, ##sett, ##e...
17    [[CLS], siren, english, police, constant, 

In [ ]:
# Mapping token to BERT vocabulary index
indexed_tokens = tokenized_text.apply(lambda sen: tokenizer.convert_tokens_to_ids(sen))

In [ ]:
# Zip words together with their indexes
token_id_tup = []
for ind, sen in enumerate(tokenized_text):
  token_id_tup.append(list(zip(sen, indexed_tokens[ind])))

In [ ]:
# Zip words together with their indexes
for tup in token_id_tup[0]:
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))


[CLS]           101
hits          4,978
low           2,659
bump         16,906
impact        4,254
[SEP]           102


In [ ]:
# Creating segment IDs for BERT to distinguish between sentences
segment_ids = []
for ind, sen in enumerate(tokenized_text):
  segment_ids.append([ind] * len(sen))

In [ ]:
tokens_tensor = [torch.tensor([token]) for token in indexed_tokens]
tokens_tensor[0]

tensor([[  101,  4978,  2659, 16906,  4254,   102]])

In [ ]:
segments_tensor = [torch.tensor([id]) for id in segment_ids]
segments_tensor[0]

tensor([[0, 0, 0, 0, 0, 0]])

In [ ]:
zipped_text_token_segment = list(zip(tokenized_text, tokens_tensor, segments_tensor))

for tup in zipped_text_token_segment:
  print(f"Sentence: {tup[0]}")
  print(f"Token_tensor: {tup[1]}")
  print(f"Segment_id: {tup[2]}")
  

Sentence: ['[CLS]', 'hits', 'low', 'bump', 'impact', '[SEP]']
Token_tensor: tensor([[  101,  4978,  2659, 16906,  4254,   102]])
Segment_id: tensor([[0, 0, 0, 0, 0, 0]])
Sentence: ['[CLS]', 'truck', 'door', 'drop', 'ground', '[SEP]']
Token_tensor: tensor([[ 101, 4744, 2341, 4530, 2598,  102]])
Segment_id: tensor([[1, 1, 1, 1, 1, 1]])
Sentence: ['[CLS]', 'body', 'falls', 'fall', 'human', 'weeds', '[SEP]']
Token_tensor: tensor([[  101,  2303,  4212,  2991,  2529, 20777,   102]])
Segment_id: tensor([[2, 2, 2, 2, 2, 2, 2]])
Sentence: ['[CLS]', 'theater', 'type', 'folding', 'seat', 'bounce', 'open', 'close', '[SEP]']
Token_tensor: tensor([[  101,  4258,  2828, 12745,  2835, 17523,  2330,  2485,   102]])
Segment_id: tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3]])
Sentence: ['[CLS]', 'applause', 'crowd', 'small', 'studio', 'audience', 'female', 'clap', 'unison', 'slow', 'tempo', 'clapping', 'crowd', '##fe', '##mal', '##e', 'crowd', '##chee', '##ring', 'indoor', '[SEP]']
Token_tensor: tensor([[  101, 20

# Evaluating the BERT model

In [ ]:
# suppressing the output with the magic command "capture"
%%capture

# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True,
                                  )

# setting model in eval mode (i.e., feed-forward operation)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# apply BERT to text and collect all hidden states form the 12 BERT layers

with torch.no_grad():

  # evaluating the model only on the first pair of token tensor and segments tensor
  outputs = model(tokens_tensor[0], segments_tensor[0])


  # in our pretrained model, third item contains hidden states form all layers
  hidden_states = outputs[2]

In [ ]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")

layer_i = 0
print ("Number of batches:", len(hidden_states[layer_i]))

batch_i = 0
print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))

token_i = 0
print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 6
Number of hidden units: 768


We want to group the values by tokens (i.e., parts of the words we want the embeddings for) and not by layer. Thus we stack all the layers together and extract the information across layers per token

In [ ]:
# Concatenate the tensors for all layers. We use `stack` here to create a new dimension in the tensor
token_embeddings = torch.stack(hidden_states, dim=0)

token_embeddings.size()

torch.Size([13, 1, 6, 768])

Current dimensions: [# layers, # batches, # tokens, # features] <br>
Desired dimensions: [# tokens, # layers, # features]

In [ ]:
# Changing the order of dimensions by permutation
# Numbers in the permute function refer to the spot in the original list, i.e., the original index in the dimension ordering in token_embedding 
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

torch.Size([6, 13, 768])

First, let’s concatenate the last four layers, giving us a single word vector per token. Each vector will have length 4 x 768 = 3,072.

In [ ]:
# stores token vectors
token_vecs_cat = []

for token in token_embeddings:

  # concatinating vectors from the last 4 model layers
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    token_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(token_vecs_cat), len(token_vecs_cat[0])))


Shape is: 6 x 3072


# Putting it all together 
Creating word embedding for entire DataFrame

In [ ]:
# apply BERT to text and collect all hidden states form the 12 BERT layers
token_vecs_cat = []


with torch.no_grad():
  # each tuple represents all the data for one of the 10 sentences
  for tup in list(zip(tokenized_text, tokens_tensor, segments_tensor)):

    # model gets tokens_tensor and segments_tensor as input, to create word embedding
    out = model(tup[1], tup[2])

  # in our pretrained model, third item contains hidden states form all layers
    hidden_states = out[2]


    # Concatenate the tensors for all layers. We use `stack` here to create a new dimension in the tensor
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)

    # stores token vectors
    for token in token_embeddings:

      # concatinating vectors from the last 4 model layers 
      # for dim=0 means only the token embeddings?
      cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

      # adding vector as a numpy array for visualization
      token_vecs_cat.append(cat_vec.numpy())

In [ ]:
print(f"Number of all word embeddings: {len(token_vecs_cat)}")

Number of all word embeddings: 637


In [ ]:
token_embeddings.size()

torch.Size([7, 13, 768])

In [ ]:
token.size()

torch.Size([13, 768])

In [ ]:
# getting all the tokens of all sentences into one list
tokens = []
for sentence in tokenized_text:
  tokens.extend(sentence)
print(f"Number of all tokens: {len(tokens)}")

Number of all tokens: 637


Showing the format of the output: The model gives us a single vector for each token, for which we printed only the 5 first dimensions

In [ ]:
token_vecs_cat[0][:5]

array([-0.56564415,  0.1800897 , -0.0054876 ,  0.01520055, -0.17841163],
      dtype=float32)

In [ ]:
# creating df form dictionary with the dictionary keys (tokens) being the rows and the values (vector dimensions) being columns
data=dict(zip(tokens, token_vecs_cat))

vocab = pd.DataFrame.from_dict(data, orient="index")
vocab

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
[CLS],-0.523729,-0.023604,-0.436968,-0.150918,-0.107386,-0.078668,0.383332,0.333220,-0.557351,-0.203414,...,-0.296992,-0.440729,-0.323404,-0.092424,0.322114,-0.276464,0.594268,-0.222647,-0.234435,0.416800
hits,0.174171,0.179587,-0.443125,0.125111,-0.167968,-0.228450,0.398193,0.109855,-0.341891,-0.413290,...,-0.487683,-0.475588,0.285095,0.742009,0.552637,-0.244662,0.547457,-0.378209,-0.592485,-0.766604
low,-0.583295,0.291757,-0.145314,-0.531971,-0.337157,-0.218252,-0.428303,0.634338,0.079402,0.290770,...,1.313356,-1.173494,0.376477,-0.689564,0.450377,1.014516,-0.370163,-0.304821,-0.312838,-0.624800
bump,-0.619284,-0.092625,0.661758,0.305355,-0.072371,0.239656,-0.190960,0.055673,-0.451952,-0.601128,...,0.584293,-0.548999,0.801620,0.125676,0.558971,0.385285,-0.989848,0.431406,-0.901355,-1.037519
impact,-0.155124,0.504677,0.484727,-0.379080,-0.408341,-0.173658,0.647599,0.064324,-0.317842,-0.462156,...,0.723333,-0.388881,0.953135,-0.271467,-0.623187,-0.257033,0.124551,0.111083,0.835148,-0.781466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
set,0.633760,-0.448184,0.606236,0.136826,-0.117163,0.538670,0.215221,0.004477,-0.265273,-0.446544,...,0.214514,0.116414,-1.224972,0.661390,-1.191625,1.110739,0.016523,-0.768467,-0.617608,-0.490599
cardiovascular,0.633049,0.419266,-0.724354,-0.330886,0.024074,0.349078,0.184914,0.185382,-0.323995,-0.279720,...,0.196417,0.333169,1.041248,0.450708,-0.531994,0.897100,-1.207147,-0.279430,-0.089463,-1.801007
exercise,1.045656,0.567426,-0.702968,-0.444921,0.170371,-0.532313,0.893823,-0.248770,-0.703019,-0.486027,...,0.356673,0.033936,0.798126,0.345628,0.689080,1.708850,-0.392543,-1.179833,-0.396895,-1.413379
jumping,0.357920,0.035868,-0.289538,0.073204,0.054859,-0.741909,0.135693,0.173989,-1.151840,0.090500,...,-0.104864,-0.067548,1.485576,-0.697580,-0.526476,0.688336,0.017144,-0.372590,0.513206,-0.690702


In [ ]:
selected_vocab = vocab[vocab.index.isin(combined_df["Word"])]
selected_vocab

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
bump,-0.619284,-0.092625,0.661758,0.305355,-0.072371,0.239656,-0.190960,0.055673,-0.451952,-0.601128,...,0.584293,-0.548999,0.801620,0.125676,0.558971,0.385285,-0.989848,0.431406,-0.901355,-1.037519
drop,0.132756,-0.078574,0.493739,-0.174659,0.200522,0.098005,-0.278988,0.157009,-0.383873,0.537640,...,0.447757,-0.411715,0.862398,-0.387089,0.610190,0.973861,-0.025759,-0.657093,-0.167273,-0.747788
fall,-0.693701,-0.082736,0.373599,-0.019570,0.029574,0.018140,0.405328,0.170272,0.147669,-0.103168,...,0.972817,-0.482178,1.313909,-0.351264,-0.319324,1.720897,-0.425010,-1.166955,-0.169317,-0.350666
bounce,-0.195936,0.243751,0.424547,-0.606902,0.119777,-0.563721,0.338499,0.525346,-0.366406,0.031399,...,0.673074,-0.923456,-0.378863,0.823578,-0.195569,0.109558,-0.307408,-1.030033,0.484827,-1.272684
clap,0.472439,0.065688,0.378112,-0.474910,-0.014752,-0.164700,-0.130745,-0.069280,-1.055036,0.498156,...,0.711284,-0.485580,-1.095746,0.854713,-0.534181,-0.084851,-1.420375,-1.232147,-0.074456,-0.734888
slam,0.346325,0.181858,0.245401,-0.350039,0.378883,-0.128553,0.431583,-0.007962,0.018592,-0.025222,...,1.734722,-0.772507,-0.613951,0.199534,-0.154212,0.341405,-0.078764,-0.133446,0.181042,-0.792536
step,0.249000,0.087933,0.762515,-0.118922,0.205954,0.448756,0.801775,-0.299156,0.105323,-0.314606,...,1.751666,-0.196120,-0.508474,-0.405685,-1.106469,0.460434,-1.687619,-0.238548,0.823814,-1.467742
punch,-0.558978,0.183725,0.510703,-0.056801,-1.051879,-0.117136,1.034639,-0.061824,-0.398490,-0.394997,...,0.220752,-0.165842,-0.128405,0.680235,-0.120511,1.504295,-0.923547,-1.108415,0.062661,-0.820820
blast,0.089465,0.437574,0.461643,-0.016037,-0.390155,-0.285903,-0.227241,0.733490,0.003230,0.053316,...,1.019733,-0.574076,-0.274089,-1.019250,-0.125812,1.077561,-0.227078,-0.951394,-0.045399,-1.092445
explode,0.362500,0.521522,0.311040,-0.057078,0.254603,-0.538439,0.122729,-0.172447,-0.282075,0.107000,...,0.364242,-0.737810,0.050291,1.062922,0.057205,-0.130714,-0.502200,-1.507880,0.564019,-1.712415


In [ ]:
tuples = [tuple(combined_df[combined_df["Word"]==i][["Category", "Word"]].values[0]) for i in selected_vocab.index]
multi_index = pd.MultiIndex.from_tuples(tuples, names=["Category", "Word"])

In [ ]:
selected_vocab = selected_vocab.set_index(multi_index)
selected_vocab.sort_index(inplace=True)

In [ ]:
selected_vocab

0         1         2         3         4     \
Category      Word                                                             
complex_sound accident     -0.634334 -0.166236  0.136587 -0.245089  0.583748   
              bowl         -0.293736 -0.248055  0.481227 -0.101880 -0.008253   
              box           0.157816  0.394526  0.487151 -0.319059  0.633125   
              destruction   0.384657  0.724283 -0.238662  0.095912 -0.153260   
              exercise      1.045656  0.567426 -0.702968 -0.444921  0.170371   
              fight        -0.659650 -0.246670  0.601151  0.038464 -0.599620   
              wrestle      -0.451489  1.022307 -0.052919 -0.040613 -0.182234   
dog           bark         -0.107780  0.767434  0.026400 -0.327552 -0.129495   
              growl         0.326138  0.074804  0.051558 -0.858733  0.854365   
              howl          0.044270  0.521488 -0.531240 -0.802673  0.140558   
              scratch       0.841389  0.345215  0.054693  0.034856  1.206289   
              whimper       0.369073 -0.022846  0.244466  0.237893  0.142363   
explosion     bang          0.113342 -0.143273  0.389491  0.173773  0.193417   
              blast         0.089465  0.437574  0.461643 -0.016037 -0.390155   
              burst         0.031339  0.038263 -0.112689  0.159344  0.138817   
              explode       0.362500  0.521522  0.311040 -0.057078  0.254603   
impact        bounce       -0.195936  0.243751  0.424547 -0.606902  0.119777   
              bump         -0.619284 -0.092625  0.661758  0.305355 -0.072371   
              clap          0.472439  0.065688  0.378112 -0.474910 -0.014752   
              drop          0.132756 -0.078574  0.493739 -0.174659  0.200522   
              fall         -0.693701 -0.082736  0.373599 -0.019570  0.029574   
              punch        -0.558978  0.183725  0.510703 -0.056801 -1.051879   
              slam          0.346325  0.181858  0.245401 -0.350039  0.378883   
              step          0.249000  0.087933  0.762515 -0.118922  0.205954   
locomotion    jump          0.438493 -0.382821  0.502864  0.060338  0.493992   
              march         0.374005  0.217147  0.697152  0.142652 -0.208766   
              move         -0.148111 -0.063348  0.171913 -0.054529  0.365095   
              run           1.312761 -0.395737  0.677469 -0.131080  0.830136   
              walk          0.345097 -0.247106  0.942014 -0.328989  0.286941   
nose          sniff         0.520344 -0.402730 -0.059111  0.203654 -0.072353   
              snort         0.464098 -0.054626 -0.038425  0.011564 -0.326761   
vehicle       accelerate    0.188334  0.244633  0.511025 -0.099088  0.941461   
              brake        -0.003465 -0.319925  0.554928 -0.554224 -0.081854   
              drive         0.672220  0.095691  0.851782 -0.212758  1.257713   
              pass          0.549774 -0.311593  0.911096 -0.260335  0.416588   
              rev           0.126325 -0.184779  1.056640  0.199307 -0.108876   
vocal         chant         0.139230  0.197055  0.179208 -0.144661 -0.452207   
              conversation -0.117645  0.266635  0.983796 -0.395504  0.472400   
              hiss          1.072044 -0.149845  0.450987 -0.125882  0.368956   
              murmur        0.136898  0.089047  0.483659 -0.275518  0.646855   
              scream        0.420526  0.208088  0.212044 -0.269599  0.225258   
              shout         0.426030  0.648306  0.558794 -0.479205  0.292200   
              talk          0.040664  0.305293  0.469462  0.116173  0.175861   
              yell          0.130285 -0.187122  0.143332 -0.241927 -0.333534   

                                5         6         7         8         9     \
Category      Word                                                             
complex_sound accident      0.537719 -0.499238  0.591444 -0.457259  0.614421   
              bowl          0.398660 -0.235137  0.595045 -0.069802 -0.446045   
              box          -0.4

In [ ]:
from scipy.spatial.distance import pdist, squareform
from scipy.stats import zscore

# cosine similarity matrix
# 1-distance matrix with the cosine distance as a metric = similarity matrix
def create_similarity_matrix_df(df, metric):
    
    matrix = 1-squareform(pdist(zscore(df, axis=0), metric))

    matrix_df = pd.DataFrame(data=matrix, columns=multi_index, index=multi_index)
    
    # matrix_df.style.set_properties(**{'text-align': 'left'})
    return matrix_df.style.background_gradient(axis=None, cmap='RdBu') # changing color

In [ ]:
create_similarity_matrix_df(selected_vocab, metric="cosine")
